In [3]:
# !pip install keras
!pip install tensorflow

^C


     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ----------------------------------- ---- 51.2/57.5 kB ? eta -:--:--
     -------------------------------------- 57.5/57.5 kB 761.8 kB/s eta 0:00:00
     ---------------------------------------- 0.0/65.5 kB ? eta -:--:--
     ---------------------------------------- 65.5/65.5 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     --------------- ------------------------ 0.6/1.5 MB 12.4 MB/s eta 0:00:01
     --------------------------------- ------ 1.2/1.5 MB 13.2 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 10.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
     -------------------------------------- 151.7/151.7 kB 8.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/284.1 MB ? eta -:--:--
   ---------------------------------------- 0.6/284.1 MB 18.8 MB/s eta 0:00:16
   ---------

In [5]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank
import librosa
import csv
from keras.layers import Conv2D,MaxPool2D,Flatten,LSTM,Dropout,Dense,TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight

